<a href="https://colab.research.google.com/github/lucianodinizrosa/lrosa/blob/master/ETL_base_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETL - carga da base Titanic (train.csv)

Nesse notebook, você encontrará o processo de ETL que usamos para carregar a base Titanic, desde uma análise exploratória preliminar até a carga propriamente dita

## Inicialização

Aqui ocorrem importações e a carga inicial dos dados para análise

In [0]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import requests

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv', sep=',', quotechar='"', quoting=True)

In [0]:
df['Name'].astype('str')

0                                Braund, Mr. Owen Harris
1      Cumings, Mrs. John Bradley (Florence Briggs Th...
2                                 Heikkinen, Miss. Laina
3           Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                               Allen, Mr. William Henry
                             ...                        
886                                Montvila, Rev. Juozas
887                         Graham, Miss. Margaret Edith
888             Johnston, Miss. Catherine Helen "Carrie"
889                                Behr, Mr. Karl Howell
890                                  Dooley, Mr. Patrick
Name: Name, Length: 891, dtype: object

## Análise exploratória

Aqui estudamos os dados para decidir que valores mantemos e que valores alteramos

In [0]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


### Renomear colunas

In [0]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [0]:
df.columns = ['passenger_id', 'survived', 'pclass', 'name', 'raw_genre', 'age', 'siblings',
       'parch', 'ticket', 'fare', 'cabin', 'boarding_port']

### Análise dos valores

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   passenger_id   891 non-null    int64  
 1   survived       891 non-null    int64  
 2   pclass         891 non-null    int64  
 3   name           891 non-null    object 
 4   raw_genre      891 non-null    object 
 5   age            714 non-null    float64
 6   siblings       891 non-null    int64  
 7   parch          891 non-null    int64  
 8   ticket         891 non-null    object 
 9   fare           891 non-null    float64
 10  cabin          204 non-null    object 
 11  boarding_port  889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [0]:
df.describe()

,passenger_id,survived,pclass,age,siblings,parch,fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [0]:
df['survived'].value_counts()

0    549
1    342
Name: survived, dtype: int64

### Filtros

Aqui vamos aprender a filtrar as linhas de acordo com critérios dos dados

In [0]:
survived_filter = df['survived'] == 1  # Uso uma série de verdadeiros e falsos para decidir quais linhas me interessam ou não

In [0]:
df[survived_filter]

,passenger_id,survived,pclass,name,raw_genre,age,siblings,parch,ticket,fare,cabin,boarding_port
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C
...,...,...,...,...,...,...,...,...,...,...,...,...
875,876,1,3,"Najib, Miss. Adele Kiamie ""Jane""",female,15.0,0,0,2667,7.2250,NaN,C
879,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C
880,881,1,2,"Shelley, Mrs. William (Imanita Parrish Hall)",female,25.0,0,1,230433,26.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S


**Importante!** Essa filtragem de dados **não** altera o dataframe original! df continua tendo todos os valores originais.

In [0]:
# Ao invés de guardar o filtro em uma variável, posso usá-lo diretamente nos colchetes:
survived_df = df[df['survived'] == 1]

A partir da linha anterior, `survived_df` passa a ser uma cópia de df **depois** de aplicado o filtro, ou seja: df continua como era antes, mas `survived_df` só contém as entradas onde a chave `'survived'` é 1

In [0]:
survived_df['pclass'].value_counts()

1    136
3    119
2     87
Name: pclass, dtype: int64

In [0]:
df[df['survived'] == 0]['pclass'].value_counts()

3    372
2     97
1     80
Name: pclass, dtype: int64

In [0]:
df['cabin']

0       NaN
1       C85
2       NaN
3      C123
4       NaN
       ... 
886     NaN
887     B42
888     NaN
889    C148
890     NaN
Name: cabin, Length: 891, dtype: object

In [0]:
def count_cabins(cabin_field):
  if isinstance(cabin_field, float) and np.isnan(cabin_field):
    return 0
  cabins = cabin_field.split(' ')
  return len(cabins)

In [0]:
df['cabin_count'] = df['cabin'].apply(count_cabins)

In [0]:
df['cabin_count'].value_counts()

0    687
1    180
2     16
3      6
4      2
Name: cabin_count, dtype: int64

In [0]:
df[df['cabin_count'] > 0]

,passenger_id,survived,pclass,name,raw_genre,age,siblings,parch,ticket,fare,cabin,boarding_port,cabin_count
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,1
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S,1
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
871,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,S,1
872,873,0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0000,B51 B53 B55,S,3
879,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C,1
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1


In [0]:
df['raw_genre'].value_counts()

male      577
female    314
Name: raw_genre, dtype: int64

In [0]:
def convert_sex(value):
  if value == 'male':
    return 1
  elif value == 'female':
    return 0
  else:
    return -1

In [0]:
df['coded_sex'] = df['raw_genre'].apply(convert_sex)

In [0]:
df['coded_sex'].value_counts()

1    577
0    314
Name: coded_sex, dtype: int64

In [0]:
genre_equivalence = {
    'female': 0,
    'male': 1,
}

In [0]:
df['raw_genre'].map(genre_equivalence)  # Outra forma de calcular o gênero codificado

0      1
1      0
2      0
3      0
4      1
      ..
886    1
887    0
888    0
889    1
890    1
Name: raw_genre, Length: 891, dtype: int64

In [0]:
df

,passenger_id,survived,pclass,name,raw_genre,age,siblings,parch,ticket,fare,cabin,boarding_port,cabin_count,coded_sex
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0,1
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1,0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,0,0
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,1,1


## Algumas agregações

Demonstração de agregações

In [0]:
df.groupby(['survived'])[['fare', 'cabin_count']].aggregate('mean')  # Agregação de média

,fare,cabin_count
survived,,
0,22.117887,0.147541
1,48.395408,0.459064


In [0]:
pd.crosstab(df['survived'], df['pclass'], normalize=True)  # Cruzamento de sobrevivência x classe do ticket

pclass,1,2,3
survived,,,
0,0.089787,0.108866,0.417508
1,0.152637,0.097643,0.133558


### Limpeza de colunas desnecessárias

In [0]:
df.drop(['name', 'raw_genre', 'ticket', 'passenger_id', 'cabin'], axis=1, inplace=True)

In [0]:
df

,survived,pclass,age,siblings,parch,fare,boarding_port,cabin_count,coded_sex
0,0,3,22.0,1,0,7.2500,S,0,1
1,1,1,38.0,1,0,71.2833,C,1,0
2,1,3,26.0,0,0,7.9250,S,0,0
3,1,1,35.0,1,0,53.1000,S,1,0
4,0,3,35.0,0,0,8.0500,S,0,1
...,...,...,...,...,...,...,...,...,...
886,0,2,27.0,0,0,13.0000,S,0,1
887,1,1,19.0,0,0,30.0000,S,1,0
888,0,3,NaN,1,2,23.4500,S,0,0
889,1,1,26.0,0,0,30.0000,C,1,1


### Salvar o arquivo de saída

In [0]:
os.mkdir('output_data')  # Criação de pasta de saída

In [0]:
df.to_csv('output_data/clean_titanic_train.csv')  # Exportação de CSV

In [0]:
df.to_excel('output_data/clean_titanic_train.xlsx')  # Exportação de planilha xlsx